In [23]:
# import the library we use to open URLs
import urllib.request
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium import webdriver
import os
import time
import io
import PIL
from PIL import Image
import hashlib
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
### using scraped data to extract more images for each celeb

# Put the path for your ChromeDriver here
DRIVER_PATH = '/home/tanmay/Desktop/scraping/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)    

In [16]:
def getobj(articleUrl):
    html= urllib.request.urlopen("http://en.wikipedia.org"+articleUrl)
    obj = BeautifulSoup(html)
    return obj

In [17]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


In [18]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = PIL.Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")


In [19]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)


In [20]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_Bollywood_actors"
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [21]:
linksN = []
namesN = []
names = []
links = []
for di in soup.findAll("div"):
    for link in di.findAll("a", href=re.compile("^(/wiki/)((?!:).)*$")):
        if('href' in link.attrs and 'title' in link.attrs and len(link.attrs) == 2):
            linksN.append(link.attrs['href'])
            namesN.append(link.attrs['title'])
[names.append(x) for x in namesN if x not in names] 
[links.append(x) for x in linksN if x not in links]
celebs = {names[i] : links[i] for i in range(len(names))}

In [9]:
cnt = 0
for celeb in celebs:
    print(celeb)
    #retrive HTML data using BeautifulSoup
    obj=getobj(celebs[celeb])
    
    #Select the right table to extract data from the celebrity page
    if obj is None:
        continue
        
    right_tables = obj.findAll('table', class_='infobox biography vcard')
    if right_tables is None:
        print(None)
        continue
    # get the image url for all the images ## for now set to one image##
    for right_table in right_tables:
        if right_table.find("img") is not None: 
            img_url = right_table.find("img")["src"]   
        else:
            continue 
    # space to add a deep learning model to detect human faces to select and download 
    # only celeb images and discard other images
    ####Start model
    
    ####End model
    
    # saving image for each celeb
    cnt = cnt + 1
    if img_url is not None:
        img_url = 'http:' + img_url
        try:
            urllib.request.urlretrieve(img_url, "/home/tanmay/Desktop/celeb_dataset/" + celeb + str(cnt) + ".jpg")
      
        except Exception as e:
            print(f"ERROR - Could not save {url} - {e}")
    ## space for adding processing steps on the image will use PILLOW in future not now!how   
    
### Saved atleast a single image for each celeb till now.


Amitabh Bachchan
Dilip Kumar
Aamir Khan
Aarun Nagar
ERROR - Could not save https://en.wikipedia.org/wiki/List_of_Bollywood_actors - <urlopen error no host given>
Abhishek Bachchan
Abhay Deol
Abhay Mahajan
Aditya Pancholi
Aditya Kumar


KeyboardInterrupt: 

In [25]:
### using scraped data to extract more images for each celeb
cnt = 0
for celeb in celebs:
    cnt += 1
    target_p = "/home/tanmay/Desktop/celeb_dataset/"
    search_and_download(search_term=celeb, driver_path=DRIVER_PATH, target_path = target_p )

Found: 100 search results. Extracting links from 0:100
Found: 5 image links, done!
SUCCESS - saved https://m.economictimes.com/thumb/msid-71641869,width-1200,height-900,resizemode-4,imgsize-706842/untitled-8.jpg - as /home/tanmay/Desktop/celeb_dataset/amitabh_bachchan/aa839da121.jpg
SUCCESS - saved https://www.orissapost.com/wp-content/uploads/2020/04/Amitabh-Bachchan-1.jpg - as /home/tanmay/Desktop/celeb_dataset/amitabh_bachchan/512cdcc69c.jpg
SUCCESS - saved https://resize.indiatvnews.com/en/resize/newbucket/715_-/2020/04/amitabh-bachchan-2-1587204201.jpg - as /home/tanmay/Desktop/celeb_dataset/amitabh_bachchan/2bc8634513.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BNTk1OTUxMzIzMV5BMl5BanBnXkFtZTcwMzMxMjI0Nw@@._V1_.jpg - as /home/tanmay/Desktop/celeb_dataset/amitabh_bachchan/512c38d1a7.jpg
SUCCESS - saved https://static.toiimg.com/photo/msid-71659006/71659006.jpg?438910 - as /home/tanmay/Desktop/celeb_dataset/amitabh_bachchan/acd4736122.jpg
Found: 200 search results. E

SUCCESS - saved https://images.indianexpress.com/2018/07/aditya-kumar-1200.jpg - as /home/tanmay/Desktop/celeb_dataset/aditya_kumar/280aa82bb1.jpg
SUCCESS - saved https://i.ytimg.com/vi/C7hXPE3UB-M/maxresdefault.jpg - as /home/tanmay/Desktop/celeb_dataset/aditya_kumar/35f6250c61.jpg
SUCCESS - saved https://images.gr-assets.com/authors/1511957592p5/17374401.jpg - as /home/tanmay/Desktop/celeb_dataset/aditya_kumar/665f8412e3.jpg
SUCCESS - saved https://img.etimg.com/thumb/width-400,height-300,msid-65402966,imgsize-/major-aditya-kumar-and-rifleman-aurangzeb-to-get-shaurya-chakra-gallantry-award.jpg - as /home/tanmay/Desktop/celeb_dataset/aditya_kumar/0b93697b79.jpg
Found: 100 search results. Extracting links from 0:100
Found: 5 image links, done!
SUCCESS - saved https://filmfare.wwmindia.com/content/2020/jan/adityaroykapur41580470473.jpg - as /home/tanmay/Desktop/celeb_dataset/aditya_roy_kapur/3c6b70a98f.jpg
SUCCESS - saved https://cdn.pinkvilla.com/files/styles/gallery-section/public/adi

SUCCESS - saved https://in.bmscdn.com/iedb/artist/images/website/poster/large/anil-kapoor-176-12-09-2017-07-42-36.jpg?1 - as /home/tanmay/Desktop/celeb_dataset/anil_kapoor/19753d808b.jpg
SUCCESS - saved https://timesofindia.indiatimes.com/thumb/msid-72082206,width-1200,height-900,resizemode-4/.jpg - as /home/tanmay/Desktop/celeb_dataset/anil_kapoor/aee5d8e98e.jpg
Found: 200 search results. Extracting links from 0:200
Found: 5 image links, done!
SUCCESS - saved https://pbs.twimg.com/profile_images/1132524486379618309/3-ClfQy3_400x400.jpg - as /home/tanmay/Desktop/celeb_dataset/aniruddh_dave/51707dc60b.jpg
SUCCESS - saved https://static.toiimg.com/thumb/msid-57357074,imgsize-138900,width-800,height-600,resizemode-75/57357074.jpg - as /home/tanmay/Desktop/celeb_dataset/aniruddh_dave/bda605ec29.jpg
SUCCESS - saved https://lh3.googleusercontent.com/proxy/uJ-x4RqlXGrPqsNXPSHT4TAVF4YxFFbwJfS1wrwOfPvySav4UAzSOJQTCixDvPK-fiIXcn4xYfHJV7lmrtEwYK4pvyR6YBsdk84VTgMYFnhcB7fm0oKGk-9vYu9Y0A - as /home/

KeyboardInterrupt: 